<a href="https://colab.research.google.com/github/Hijuli66/33/blob/master/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install streamlit pyngrok sentence-transformers keybert -q
!pip install -q torchxrayvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 51.9 MB/s eta 0:00:00


In [27]:
from pyngrok import ngrok
!ngrok authtoken 36xpcdxUNo8UH0GzDh6G7E3pugM_7PNih33G5fa7d9A9hXKMF

!streamlit run app.py &>/dev/null&
public_url = ngrok.connect(8501)
print("访问地址：", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
访问地址： NgrokTunnel: "https://carditic-stephine-nonaccredited.ngrok-free.dev" -> "http://localhost:8501"


In [15]:
# 先挂载 Google Drive（如果还没挂载的话）
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
from pyngrok import ngrok

# 杀掉所有现有的 ngrok 隧道
ngrok.kill()

# 重新启动 Streamlit（如果被杀了的话）
!pkill -f streamlit
!streamlit run app.py --server.port=8501 --server.address=0.0.0.0 &>/dev/null &

# 等待启动
import time
time.sleep(10)

# 只创建一个新隧道
public_url = ngrok.connect(8501, bind_tls=True)
print("新的公网访问地址：", public_url)

新的公网访问地址： NgrokTunnel: "https://carditic-stephine-nonaccredited.ngrok-free.dev" -> "http://localhost:8501"


In [26]:
%%writefile app.py
# ==================== app.py ====================
import streamlit as st
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import cv2
import os
import re
import torchxrayvision as xrv
import torchvision.transforms as transforms   # 如果还没导入的话，加上这行

# ------------------- 配置 -------------------
st.set_page_config(page_title="胸部X光AI辅助诊断系统", layout="wide")
st.title("胸部X光肺炎AI辅助诊断系统")
st.markdown("""
### 请上传胸部X光图像和对应的放射科报告
系统将实时提供：
- Grad-CAM 热力图（高亮肺炎可疑区域）
- 报告关键词自动提取与高亮
- 图文匹配相似度 + AI 诊断建议
""")

# ------------------- 项目路径 -------------------
PROJECT_PATH = '/content/drive/MyDrive/33'
MOBILENET_PATH = f'{PROJECT_PATH}/Images/Models/mobilenetv3_final.pth'
CONTRASTIVE_PATH = f'{PROJECT_PATH}/Matching/best_contrastive_model.pth'


@st.cache_resource
def load_models():
    st.info("正在加载所有模型（MobileNetV3 + 图文匹配 + 肺部分割），首次运行需60-90秒...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ==================== 1. MobileNetV3 分类模型 ====================
    mobilenet = models.mobilenet_v3_small(pretrained=False)
    mobilenet.features[0][0] = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1, bias=False)
    num_features = mobilenet.classifier[3].in_features
    mobilenet.classifier[3] = nn.Linear(num_features, 2)

    state_dict = torch.load(MOBILENET_PATH, map_location=device)
    model_dict = mobilenet.state_dict()
    filtered_dict = {k: v for k, v in state_dict.items() if k in model_dict and v.shape == model_dict[k].shape}
    mobilenet.load_state_dict(filtered_dict, strict=False)
    mobilenet.to(device)
    mobilenet.eval()
    final_conv_layer = mobilenet.features[-1]

    # ==================== 2. 图文对比模型 ====================
    class ContrastiveModel(nn.Module):
        def __init__(self, img_dim=576, text_dim=768, embed_dim=256):
            super().__init__()
            self.proj_img = nn.Linear(img_dim, embed_dim)
            self.proj_text = nn.Linear(text_dim, embed_dim)

        def forward(self, img=None, text=None):
            if img is not None:
                return F.normalize(self.proj_img(img), dim=-1)
            if text is not None:
                return F.normalize(self.proj_text(text), dim=-1)

    contrastive_model = ContrastiveModel()
    contrastive_model.load_state_dict(torch.load(CONTRASTIVE_PATH, map_location=device))
    contrastive_model.to(device)
    contrastive_model.eval()

    # ==================== 3. 文本编码器 ====================
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer('shibing624/text2vec-base-chinese')
    embedder.to(device)

    # ==================== 4. 肺部分割模型 ====================
    lung_seg_model = xrv.baseline_models.chestx_det.PSPNet()
    lung_seg_model.to(device)
    lung_seg_model.eval()

    # 删除有问题的transform定义，替换为：
    lung_transform = None  # 不使用transform，直接在函数中处理


    st.success("所有模型加载完成！")

    return (device, mobilenet, final_conv_layer, contrastive_model,
            embedder, lung_seg_model, lung_transform)

# ------------------- 关键词提取函数 -------------------
def extract_keywords(text, top_n=20):
    if not text or not isinstance(text, str):
        return []
    text = text.strip()
    if len(text) == 0:
        return []

    keywords = set()
    text_clean = text.replace(' ', '')

    basic_patterns = [
        r'磨玻璃影',r'肺炎',
        r'双肺', r'肺野', r'纹理', r'斑片影', r'条索影',
        r'发热', r'咳嗽', r'干咳', r'咳痰'
    ]
    for p in basic_patterns:
        keywords.update(re.findall(p, text_clean))

    symptoms = re.findall(r'(乏力|肌肉酸痛|咽痛|胸闷|呼吸困难|气促|头痛|恶心|呕吐|腹泻)', text_clean)
    keywords.update(symptoms)

    epi_history = re.findall(r'(疫区旅居史|新冠患者接触史|密切接触史)', text_clean)
    keywords.update(epi_history)

    vaccine = re.findall(r'接种新冠疫苗\s*\d+剂', text_clean)
    keywords.update(vaccine)

    nucleic = re.findall(r'(新冠病毒核酸|核酸)\s*(阳性|阴性)', text_clean)
    antibody = re.findall(r'(新冠病毒抗体|抗体)\s*(阳性|阴性)', text_clean)
    for a, b in (nucleic + antibody):
        keywords.add(f"{a}{b}")

    img_short_patterns = [
        r'双肺多发磨玻璃影', r'[左右][上下]肺[^。；,]*?实变', r'肺野清晰', r'无明显异常',
        r'双肺纹理增多', r'斑片影', r'条索影', r'心影正常', r'肺门影未见增大',
        r'肺部通气良好', r'无炎症影'
    ]
    for p in img_short_patterns:
        keywords.update(re.findall(p, text_clean))

    ct_long_patterns = [
        r'胸部CT[^。；,]*?双肺[^。；,]*?磨玻璃影[^。；,]*?',
        r'[左右][上下]肺[^。；,]*?炎症性改变[^。；,]*?',
        r'右肺下叶实变[^。；,]*?',
        r'肺野清晰[^。；,]*?心影正常[^。；,]*?无异常密度影',
        r'X光胸片[^。；,]*?双肺纹理正常[^。；,]*?肺门影未见增大',
        r'肺部通气良好[^。；,]*?无炎症影'
    ]
    for p in ct_long_patterns:
        keywords.update(re.findall(p, text_clean))

    diag_patterns = [
        r'新型冠状病毒肺炎', r'病毒性肺炎', r'社区获得性肺炎', r'新冠肺炎',
        r'轻型|普通型|重型|危重型'
    ]
    for p in diag_patterns:
        keywords.update(re.findall(p, text_clean))

    result = list(keywords)
    result.sort(key=len, reverse=True)
    return result[:top_n]

# ------------------- Grad-CAM 函数 -------------------
def generate_gradcam(image_pil, model, target_layer):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])

    input_tensor = preprocess(image_pil).unsqueeze(0).to(device)

    original_mode = model.training
    model.train()

    for param in model.parameters():
        param.requires_grad = True

    activations = None
    gradients = None

    def forward_hook(module, input, output):
        nonlocal activations
        activations = output
        activations.retain_grad()

    def backward_hook(module, grad_input, grad_output):
        nonlocal gradients
        gradients = grad_output[0]

    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_backward_hook(backward_hook)

    try:
        model.zero_grad()

        with torch.set_grad_enabled(True):
            output = model(input_tensor)
            pred_class = output.argmax(dim=1).item()
            score = output[0, pred_class]
            score.backward()

        if gradients is None:
            print("警告: 未通过hook获取梯度，尝试备用方法...")
            model.zero_grad()
            one_hot = torch.zeros_like(output)
            one_hot[0, pred_class] = 1.0
            with torch.set_grad_enabled(True):
                output = model(input_tensor)
                output.backward(gradient=one_hot)
            if activations.grad is not None:
                gradients = activations.grad

        grads_val = gradients.detach().cpu().numpy().squeeze()
        acts_val = activations.detach().cpu().numpy().squeeze()
        weights = np.mean(grads_val, axis=(1, 2))

        cam = np.zeros(acts_val.shape[1:], dtype=np.float32)
        for i, w in enumerate(weights):
            cam += w * acts_val[i]

        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, (image_pil.width, image_pil.height))
        cam = cam - np.min(cam)
        cam = cam / (np.max(cam) + 1e-8)

        print(f"Grad-CAM生成完成，尺寸: {cam.shape}")

                # ===== 应用肺部掩码（正确位置和缩进）=====
        try:
            lung_mask = generate_lung_mask(image_pil)  # 返回 (H, W) float32 numpy array

            # 如果尺寸不匹配，调整掩码大小
            if lung_mask.shape != cam.shape:
                lung_mask = cv2.resize(lung_mask, (cam.shape[1], cam.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)

            # 轻微膨胀掩码，使边界过渡更自然
            kernel = np.ones((25, 25), np.uint8)
            lung_mask_dilated = cv2.dilate(lung_mask, kernel, iterations=1)

            # 软融合：主要用原掩码 + 轻微扩展
            combined_mask = np.maximum(lung_mask, lung_mask_dilated * 0.4)

            # 应用到热力图
            cam = cam * combined_mask

            # 重新归一化
            cam = np.maximum(cam, 0)
            if cam.max() > 0:
                cam = cam / cam.max()

            st.caption("✔ 已成功应用肺部分割掩码，热力图仅在肺部区域高亮")

        except Exception as e:
            st.warning(f"肺部掩码应用失败，使用原始热力图: {e}")

    finally:
        forward_handle.remove()
        backward_handle.remove()
        model.train(original_mode)

    return cam
def generate_lung_mask(image_pil):
    """
    高质量肺部掩码生成函数（修复版本）
    """
    try:
        # 将PIL图像转换为numpy数组
        img_np = np.array(image_pil)

        # 如果图像是彩色，转换为灰度
        if len(img_np.shape) == 3:
            img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

        # 确保是uint8类型（0-255）
        if img_np.dtype != np.uint8:
            img_np = (img_np * 255).astype(np.uint8) if img_np.max() <= 1.0 else img_np.astype(np.uint8)

        # 中心裁剪成正方形（使用与之前相同的逻辑）
        def center_crop_to_square(img_np):
            h, w = img_np.shape[:2]
            min_side = min(h, w)
            start_h = (h - min_side) // 2
            start_w = (w - min_side) // 2
            return img_np[start_h:start_h+min_side, start_w:start_w+min_side]

        img_square = center_crop_to_square(img_np)

        # **修复：正确的数据预处理方式**
        # 1. 归一化到[0,1]（使用xrv.datasets.normalize）
        img_normalized = xrv.datasets.normalize(img_square, 255)

        # 2. 转换为torch张量
        img_tensor = torch.from_numpy(img_normalized).float().unsqueeze(0).unsqueeze(0)

        # 3. 调整大小到512x512
        img_tensor = torch.nn.functional.interpolate(
            img_tensor,
            size=(512, 512),
            mode='bilinear',
            align_corners=False
        ).to(device)

        # 4. 推理获取分割结果
        with torch.no_grad():
            output = lung_seg_model(img_tensor)
            prob = torch.softmax(output, dim=1)

        # 5. 提取左右肺概率（类别4和5）
        lung_prob = torch.max(prob[0, [4, 5]], dim=0)[0].cpu().numpy()

        # 6. 阈值化获取掩码
        mask_512 = (lung_prob > 0.4).astype(np.uint8)

        # 7. 形态学操作改善掩码
        mask_512 = cv2.morphologyEx(mask_512, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8), iterations=3)
        mask_512 = cv2.morphologyEx(mask_512, cv2.MORPH_OPEN, np.ones((5,5), np.uint8), iterations=1)

        # 8. 调整掩码到原始图像尺寸
        mask_resized = cv2.resize(
            mask_512,
            (img_np.shape[1], img_np.shape[0]),
            interpolation=cv2.INTER_NEAREST
        )

        # 9. 转换为float32类型（用于后续乘法操作）
        mask_resized = mask_resized.astype(np.float32)

        return mask_resized

    except Exception as e:
        st.error(f"肺部掩码生成失败: {str(e)}")
        # 返回全1掩码作为备选
        return np.ones_like(np.array(image_pil).astype(np.float32))


# ------------------- 用户上传及后续逻辑 -------------------
col1, col2 = st.columns(2)
with col1:
    uploaded_image = st.file_uploader("上传胸部X光图像", type=['jpg', 'png', 'jpeg'])
with col2:
    uploaded_report_file = st.file_uploader("上传放射科报告（.txt）", type=['txt'])
    report_text = st.text_area("或者直接在此粘贴报告文本", height=200)

if uploaded_report_file is not None:
    report_text = uploaded_report_file.read().decode("utf-8")

if uploaded_image is not None and report_text.strip() != "":
    # X光图转为灰度（与训练一致）
    image_pil = Image.open(uploaded_image).convert('L')
    device, mobilenet, final_conv_layer, contrastive_model, embedder, lung_seg_model, lung_transform = load_models()

    with st.spinner("正在分析图像与报告，请稍候..."):
        # 1. 先进行 Grad-CAM（需要梯度）
        try:
            cam = generate_gradcam(image_pil, mobilenet, final_conv_layer)
            cam_color = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
            cam_color = cv2.cvtColor(cam_color, cv2.COLOR_BGR2RGB)
            overlay = cv2.addWeighted(np.array(image_pil.convert('RGB')), 0.65, cam_color, 0.35, 0)
            overlay_pil = Image.fromarray(overlay)

            cam_success = True
        except Exception as e:
            st.error(f"Grad-CAM生成失败: {e}")
            overlay_pil = image_pil.convert('RGB')
            cam_success = False

        # 2. 再进行图文相似度计算（不需要梯度）
        with torch.no_grad():
            # 确保模型在eval模式
            mobilenet.eval()
            contrastive_model.eval()

            # 文本嵌入 - 使用 no_grad
            text_emb = embedder.encode(report_text, convert_to_tensor=True, device=device).unsqueeze(0)

            # 关键修复：确保文本嵌入不在推理模式下
            if hasattr(text_emb, 'is_inference') and text_emb.is_inference:
                text_emb = text_emb.clone().requires_grad_(False)

            # 图像特征提取
            preprocess = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485], std=[0.229])
            ])
            img_tensor = preprocess(image_pil).unsqueeze(0).to(device)

            # 获取图像特征
            img_feat = mobilenet.features(img_tensor)
            img_feat = F.adaptive_avg_pool2d(img_feat, (1, 1)).view(1, -1)

            # 投影到对比空间
            try:
                img_proj = contrastive_model(img=img_feat)
                text_proj = contrastive_model(text=text_emb)
                similarity = F.cosine_similarity(img_proj, text_proj).item()
                similarity = max(0.0, min(1.0, (similarity + 1) / 2))  # 归一化到0-1
            except Exception as e:
                st.error(f"图文匹配计算失败: {e}")
                similarity = 0.5

            # 关键词高亮
            keywords = extract_keywords(report_text, top_n=15)
            highlighted_report = report_text
            for kw in keywords:
                if len(kw) > 1 and kw in highlighted_report:
                    highlighted_report = highlighted_report.replace(
                        kw, f'<mark style="background-color:#FFF59D;padding:3px;border-radius:3px">{kw}</mark>'
                    )

    # 显示结果
    st.success("分析完成！")

    col1, col2 = st.columns(2)
    with col1:
        st.image(image_pil.convert('RGB'), caption="原始X光图像", width='stretch')
        st.image(overlay_pil, caption="红色区域表示模型关注的可疑肺炎区域", width='stretch')

    with col2:
        st.markdown("### 放射科报告")
        st.markdown(highlighted_report, unsafe_allow_html=True)

        st.markdown("### 🧠 AI 多模态辅助分析结论")

        with torch.no_grad():
            output = mobilenet(img_tensor)
            prob = F.softmax(output, dim=1)[0]
            pneumonia_prob = prob[1].item()

        if pneumonia_prob > 0.7:
            diagnosis = "肺炎疑似阳性"; color = "🔴"; st.info("📊 **肺炎概率较高，建议临床重点关注**")
        elif pneumonia_prob < 0.4:
            diagnosis = "肺炎疑似阴性"; color = "🟢"; st.info("📊 **倾向于正常表现**")
        else:
            diagnosis = "肺炎可能性不确定"; color = "🟡"; st.info("📊 **需结合临床进一步判断**")

        st.markdown(f"**{color} AI 辅助建议：{diagnosis}**")
        st.metric(label="图像分类置信度（肺炎概率）", value=f"{pneumonia_prob:.1%}")

        with st.expander("图文多模态一致性评估"):
            st.markdown("系统评估图像与报告描述的语义一致性（开发持续优化中）")
            level = "较高" if similarity > 0.7 else "中等" if similarity > 0.5 else "较低"
            st.metric("图文语义一致性分数", f"{similarity:.4f}")
            st.write(f"一致性水平：**{level}**")

        st.caption("本系统仅供临床辅助参考，不替代医生诊断")

else:
    if uploaded_image is None:
        st.warning("请上传胸部X光图像")
    if report_text.strip() == "":
        st.warning("请上传或粘贴放射科报告文本")


Overwriting app.py
